In [1]:
import os
import pandas as pd

csv_path = r"D:\infosys project\Dataset\Dataset\Recordings\audio__details.csv"
audio_folder_path = r"D:\infosys project\Dataset\Dataset\Recordings\Train"


df = pd.read_csv(csv_path)
    

df['File_name'] = df['File_name'].apply(lambda x: os.path.abspath(os.path.join(audio_folder_path, os.path.basename(x))))


df.to_csv(csv_path, index=False)

In [8]:
import pandas as pd

# Load CSV file
data = pd.read_csv(r"D:\infosys project\Dataset\Dataset\Recordings\audio__details.csv")


In [9]:
from sklearn.model_selection import train_test_split

# Splitting data: 80% for training, 20% for evaluation
train_data, eval_data = train_test_split(data, test_size=0.2, random_state=42)


In [10]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_data)
eval_dataset = Dataset.from_pandas(eval_data)


In [2]:
from datasets import load_dataset, Audio


dataset = load_dataset('csv', data_files=r"D:\infosys project\Dataset\Dataset\Recordings\audio__details.csv")

dataset = dataset.cast_column('File_name', Audio(sampling_rate=16000))

dataset = dataset.rename_column('File_name', 'audio')
dataset = dataset.rename_column('phrase', 'sentence')

C:\Users\lenovo\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 108 examples [00:00, 3275.50 examples/s]


In [3]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-base")

def prepare_dataset(batch):
    # Process audio to input features
    batch["input_features"] = processor(batch["audio"]["array"], sampling_rate=16000).input_features[0]
    # Process transcription to labels
    batch["labels"] = processor.tokenizer(batch["sentence"]).input_ids
    return batch

# Specify only the columns you need to keep
columns_to_remove = [
    'audio_clipping', 'audio_clipping:confidence', 'background_noise_audible', 
    'background_noise_audible:confidence', 'overall_quality_of_the_audio', 
    'quiet_speaker', 'quiet_speaker:confidence', 'speaker_id', 'file_download', 
    'prompt', 'writer_id'
]

# Filter out columns that are not in the dataset
columns_to_remove = [col for col in columns_to_remove if col in dataset.column_names]

# Apply the processing function to the dataset
dataset = dataset.map(prepare_dataset, remove_columns=columns_to_remove)


Map: 100%|██████████| 108/108 [00:23<00:00,  4.54 examples/s]


In [17]:
import torch
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from torch.nn.utils.rnn import pad_sequence


model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base")


training_args = Seq2SeqTrainingArguments(
    output_dir="whisper-finetuned",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    num_train_epochs=3,
    fp16=False,
    save_steps=100,
    logging_steps=10,
    remove_unused_columns=False,
    evaluation_strategy="steps",
    eval_steps=100,
    save_total_limit=2,
)

In [5]:
class DataCollatorForWhisper:
    def __call__(self, features):
        
        input_features = [torch.tensor(feature["input_features"]) for feature in features]
        labels = [torch.tensor(feature["labels"]) for feature in features]

        
        input_features_padded = pad_sequence(input_features, batch_first=True, padding_value=0)
        labels_padded = pad_sequence(labels, batch_first=True, padding_value=-100)

        return {
            "input_features": input_features_padded,
            "labels": labels_padded
        }

In [6]:
data_collator = DataCollatorForWhisper()

In [18]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor.tokenizer,
    data_collator=data_collator,
)


C:\Users\lenovo\AppData\Local\Temp\ipykernel_14780\1972757673.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [19]:
trainer.train()

  0%|          | 0/6 [00:00<?, ?it/s]

KeyError: 'input_features'

In [9]:
model.save_pretrained("whisper-finetuned1")
processor.save_pretrained("whisper-finetuned1")

[]